In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
# from langchain_text_splitters import NLTKTextSplitter, SpacyTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (BSHTMLLoader, 
                                                  DirectoryLoader, 
                                                  Docx2txtLoader, 
                                                  NewsURLLoader, 
                                                  PyPDFLoader, 
                                                  PyMuPDFLoader,
                                                  MathpixPDFLoader,
                                                  RecursiveUrlLoader, 
                                                  SeleniumURLLoader, 
                                                  TextLoader, 
                                                  UnstructuredHTMLLoader,
                                                UnstructuredImageLoader,
                                                UnstructuredPowerPointLoader, 
                                                UnstructuredURLLoader, 
                                                UnstructuredWordDocumentLoader, 
                                                YoutubeLoader)
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
import pymupdf

import os
import torch
import re
from PIL import Image
from natsort import natsorted
import shutil
from transformers import NougatProcessor, VisionEncoderDecoderModel, pipeline
from src.utils import *

In [3]:
device = take_device()

Selected device: cuda


In [4]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [5]:
# embeddings=HuggingFaceEmbeddings(model_name=model_config.get('embedding_model'),
#                                         model_kwargs={'device':take_device()})

In [6]:
pdf_path = "data/web_data/introduction to algo4 new.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text = combine_short_doc(ori_text, 100)

In [7]:
ori_text[22]

Document(page_content='8 \nChapter 1 The Role of Algorithms in Computing \n\ue001 \nGiven a mechanical design in terms of a library of parts, where each part may \ninclude instances of other parts, list the parts in order so that each part appears \nbefore any part that uses it. If the design comprises n parts, then there are nŠ \npossible orders, where nŠ denotes the factorial function. Because the factorial \nfunction grows faster than even an exponential function, you cannot feasibly \ngenerate each possible order and then verify that, within that order, each part \nappears before the parts using it (unless you have only a few parts). This prob- \nlem is an instance of topological sorting, and Chapter 20 shows how to solve \nthis problem efûciently. \n\ue001 \nA doctor needs to determine whether an image represents a cancerous tumor or \na benign one. The doctor has available images of many other tumors, some of \nwhich are known to be cancerous and some of which are known to be ben

In [8]:
# for x in dir(loader):
#     if not x.startswith("_"):
#         print(x)

In [9]:
class MathEquation:
    def __init__(self, num1, num2, operator):
        pass

In [10]:
ori_text[10]

Document(page_content='xiv \nPreface \nThe text is intended primarily for use in undergraduate or graduate courses in \nalgorithms or data structures. Because it discusses engineering issues in algorithm \ndesign, as well as mathematical aspects, it is equally well suited for self-study by \ntechnical professionals. \nIn this, the fourth edition, we have once again updated the entire book. The \nchanges cover a broad spectrum, including new chapters and sections, color illus- \ntrations, and what we hope you’ll ûnd to be a more engaging writing style. \nTo the teacher \nWe have designed this book to be both versatile and complete. You should ûnd it \nuseful for a variety of courses, from an undergraduate course in data structures up \nthrough a graduate course in algorithms. Because we have provided considerably \nmore material than can ût in a typical one-term course, you can select the material \nthat best supports the course you wish to teach. \nYou should ûnd it easy to organize yo

In [11]:
def is_latex_format(doc):
    # Check if doc.metadata exists and is a dictionary
    if not hasattr(doc, 'metadata') or not isinstance(doc.metadata, dict):
        return False
    
    # Check for "tex" in any metadata value (case-insensitive)
    return any("tex" in str(value).lower() for value in doc.metadata.values())


print(is_latex_format(ori_text[10]))           

False


In [12]:
text_list = [txt.page_content for txt in ori_text]

In [13]:
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification", model="CrissWang/bert-math", torch_dtype=torch.float16, device = take_device())

Selected device: cuda


In [14]:
outputs = pipe(text_list, batch_size=8, truncation="only_first",)
print(outputs)

[{'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 0.9921875}, {'label': 'Not Math', 'score': 0.994140625}, {'label': 'Not Math', 'score': 0.998046875}, {'label': 'Not Math', 'score': 0.986328125}, {'label': 'Not Math', 'score': 0.97607421875}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0

In [15]:
# for input, output in zip(text_list, outputs):
#     print(f"output: {output['label']} - Input: {input}")
#     print("---------------")

In [16]:
math_indices = [index for index, output in enumerate(outputs) if output['label'].lower() == "math"]

# Print the list of indices where the label is "math"
print(math_indices)

[34, 44, 45, 50, 51, 54, 56, 57, 68, 70, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 96, 97, 101, 105, 110, 113, 123, 124, 125, 126, 127, 128, 132, 133, 135, 147, 156, 161, 162, 178, 181, 184, 189, 198, 203, 210, 223, 227, 229, 237, 244, 247, 250, 257, 260, 265, 293, 301, 308, 315, 327, 345, 351, 360, 363, 375, 380, 383, 391, 392, 400, 406, 408, 409, 412, 416, 417, 434, 448, 455, 463, 469, 470, 481, 501, 513, 518, 521, 532, 542, 543, 545, 547, 559, 566, 575, 577, 601, 602, 604, 610, 622, 624, 627, 630, 636, 638, 640, 643, 644, 651, 660, 661, 662, 663, 666, 667, 669, 673, 674, 675, 687, 688, 689, 690, 692, 693, 694, 696, 699, 700, 704, 715, 716, 719, 722, 734, 736, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 770, 780, 785, 787, 789, 795, 805, 808, 828, 830, 831, 832, 834, 836, 837, 839, 843, 844, 847, 850, 851, 852, 854, 856, 862, 864, 871, 873, 875, 877, 878, 879, 882, 886, 889, 890, 891, 892, 894, 895, 896, 897, 898, 899, 900, 901, 908, 909, 914, 915, 916, 917, 922, 923, 924

In [17]:
print(len(text_list), len(math_indices))


1297 294


In [18]:
doc = pymupdf.open(pdf_path)  # open document
tmp_path = "data/tmp/"
for page in doc:  # iterate through the pages
    if not (page.number in math_indices):
        continue
    pix = page.get_pixmap()  # render page to an image
    pix.save(f"{tmp_path}/%i.png" % page.number)  # store image as a PNG

In [19]:
nougat_processor = pipeline("image-to-text", model="facebook/nougat-small",  device = take_device(), max_new_tokens=1024 )

Selected device: cuda


In [20]:
def get_all_dir(root_dir):
    # Get the list of items in the directory
    items = os.listdir(root_dir)
    # Sort the items list in natural order
    items = natsorted(items)
    # Create full paths
    image_list = [os.path.join(root_dir, item) for item in items]
    return image_list

image_list = get_all_dir(tmp_path)
print(image_list)

['data/tmp/34.png', 'data/tmp/44.png', 'data/tmp/45.png', 'data/tmp/50.png', 'data/tmp/51.png', 'data/tmp/54.png', 'data/tmp/56.png', 'data/tmp/57.png', 'data/tmp/68.png', 'data/tmp/70.png', 'data/tmp/75.png', 'data/tmp/76.png', 'data/tmp/77.png', 'data/tmp/78.png', 'data/tmp/79.png', 'data/tmp/80.png', 'data/tmp/81.png', 'data/tmp/82.png', 'data/tmp/83.png', 'data/tmp/84.png', 'data/tmp/85.png', 'data/tmp/86.png', 'data/tmp/96.png', 'data/tmp/97.png', 'data/tmp/101.png', 'data/tmp/105.png', 'data/tmp/110.png', 'data/tmp/113.png', 'data/tmp/123.png', 'data/tmp/124.png', 'data/tmp/125.png', 'data/tmp/126.png', 'data/tmp/127.png', 'data/tmp/128.png', 'data/tmp/132.png', 'data/tmp/133.png', 'data/tmp/135.png', 'data/tmp/147.png', 'data/tmp/156.png', 'data/tmp/161.png', 'data/tmp/162.png', 'data/tmp/178.png', 'data/tmp/181.png', 'data/tmp/184.png', 'data/tmp/189.png', 'data/tmp/198.png', 'data/tmp/203.png', 'data/tmp/210.png', 'data/tmp/223.png', 'data/tmp/227.png', 'data/tmp/229.png', 'da

In [21]:
outputs = nougat_processor(image_list, batch_size=8, max_new_tokens=512,)

In [26]:
outputs[:5]

[[{'generated_text': "For a concrete example, let us pit a faster computer (computer A) running insertion sort against a slower computer (computer B) running merge sort. They each must sort an array of 10 million numbers. (Although 10 million numbers might seem like a lot, if the numbers are eight-byte integers, then the input occupies about 80 megabytes, which fits in the memory of even an inexpensive laptop computer many times over.) Suppose that computer A executes 10 billion instructions per second (faster than any single sequential computer at the time of this writing) and computer B executes only 10 million instructions per second (much slower than most contemporary computers), so that computer A is 1000 times faster than computer B in raw computing power. To make the difference even more dramatic, suppose that the world's crafts programmer codes insertion sort in machine language for computer A, and the resulting code requires \\(2n^{2}\\) instructions to sort \\(n\\) numbers. S

In [23]:
import shutil
def remove_and_recreate_folder(folder_path):
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Remove the folder and all its contents
        shutil.rmtree(folder_path)
        print(f"Removed the folder: {folder_path}")
    
    # Recreate the folder
    os.makedirs(folder_path)
    print(f"Recreated the folder: {folder_path}")

remove_and_recreate_folder(tmp_path)

Removed the folder: data/tmp/
Recreated the folder: data/tmp/


In [24]:
for i, idx in enumerate(math_indices):
    ori_text[idx].page_content = outputs[i][0]["generated_text"]

In [27]:
ori_text[:5]

[Document(page_content='  Introduction to Algorithms \nFourth Edition \n  Thomas H. Cormen \nCharles E. Leiserson \nRonald L. Rivest \nClifford Stein \nIntroduction to Algorithms \nFourth Edition \nThe MIT Press \nCambridge, Massachusetts \nLondon, England \n', metadata={'source': 'data/web_data/introduction to algo4 new.pdf', 'file_path': 'data/web_data/introduction to algo4 new.pdf', 'page': 4, 'total_pages': 1312, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}),
 Document(page_content='c \n\ue001 2022 Massachusetts Institute of Technology \nAll rights reserved. No part of this book may be reproduced in any form or by any electronic or mechanical means \n(including photocopying, recording, or information storage and retrieval) without permission in writing from the \npublisher. \nThe MIT Press would like to thank the anonymous peer reviewers who provided comments on drafts